## 2. Datos

### Conjunto de datos de Finanzas

Este conjunto de datos contiene información sobre reservas de hoteles. A continuación, tenemos una descripción de lo que nos podemos encontrar en el conjunto de datos:

| Columna                       | Descripción                                                                                   |
|-------------------------------|-----------------------------------------------------------------------------------------------|
| `hotel`                       | Tipo de hotel                                                                                 |
| `is_canceled`                 | Indica si la reserva fue cancelada (`True`) o no (`False`)                                    |
| `lead_time`                   | Número de días entre la fecha de reserva y la fecha de llegada al hotel                       |
| `arrival_date_year`           | Año de llegada al hotel                                                                       |
| `arrival_date_month`          | Mes de llegada al hotel                                                                       |
| `arrival_date_week_number`    | Número de la semana de llegada al hotel                                                       |
| `arrival_date_day_of_month`   | Día del mes de llegada al hotel                                                               |
| `stays_in_weekend_nights`     | Número de noches que el cliente se quedó durante el fin de semana                             |
| `stays_in_week_nights`        | Número de noches que el cliente se quedó durante la semana                                    |
| `adults`                      | Número de adultos que acompañaban al cliente en la reserva                                    |
| `children`                    | Número de niños que acompañaban al cliente en la reserva                                      |
| `babies`                      | Número de bebés que acompañaban al cliente en la reserva                                      |
| `meal`                        | Tipo de comida incluida en la reserva (`BB`: Desayuno, `HB`: Media Pensión, `FB`: Pensión Completa) |
| `country`                     | País de origen del cliente                                                                    |
| `market_segment`              | Segmento de mercado al que pertenece la reserva                                               |
| `distribution_channel`        | Canal de distribución utilizado para realizar la reserva                                      |
| `is_repeated_guest`           | Indica si el cliente es un huésped repetido (`1`) o no (`0`)                                  |
| `previous_cancellations`      | Número de reservas canceladas por el cliente antes de esta reserva                            |
| `previous_bookings_not_canceled` | Número de reservas no canceladas por el cliente antes de esta reserva                    |
| `reserved_room_type`          | Tipo de habitación reservada                                                                  |
| `assigned_room_type`          | Tipo de habitación asignada en la reserva                                                     |
| `booking_changes`             | Número de cambios realizados en la reserva                                                    |
| `agent`                       | Identificador del agente involucrado en la reserva                                            |
| `company`                     | Identificador de la compañía involucrada en la reserva                                        |
| `days_in_waiting_list`        | Número de días que la reserva estuvo en lista de espera                                       |
| `customer_type`               | Tipo de cliente que realizó la reserva (`Transient`, `Contract`, `Group`, `Transient-Party`)  |
| `adr`                         | Tarifa promedio diaria pagada por la reserva                                                  |
| `required_car_parking_spaces` | Número de espacios de estacionamiento requeridos por el cliente                               |
| `total_of_special_requests`   | Número total de solicitudes especiales realizadas por el cliente                              |
| `reservation_status`          | Estado de la reserva (`Check-Out`: Salida, `Canceled`: Cancelada)                             |
| `reservation_status_date`     | Fecha del estado de la reserva                                                                |


In [1]:
# Libraries 
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re
from word2number import w2n
from itertools import combinations
from scipy.stats import kstest, spearmanr, pearsonr, chi2_contingency

# Configuration
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns


In [19]:
df = pd.read_csv('data/finanzas-hotel-bookings.csv', index_col=0)

/var/folders/dc/5lb7jh9x1d317m9p99y8hjm40000gn/T/ipykernel_20673/2578282623.py:1: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/finanzas-hotel-bookings.csv', index_col=0)


In [21]:
df.head(3)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN


In [18]:
df.isna().sum()

hotel                              63487
is_canceled                        63487
lead_time                          63387
arrival_date_year                 118048
arrival_date_month                 63487
arrival_date_week_number           81873
arrival_date_day_of_month          63606
stays_in_weekend_nights            63387
stays_in_week_nights               63387
adults                             63449
children                          113575
babies                             63446
meal                               63487
country                           117823
market_segment                    122943
distribution_channel               76978
is_repeated_guest                  68501
previous_cancellations            106649
previous_bookings_not_canceled     63487
reserved_room_type                103602
assigned_room_type                 63487
booking_changes                    63459
agent                              79827
company                           179007
days_in_waiting_

In [15]:
df.is_canceled.unique()

array([False, True, nan], dtype=object)

In [14]:
df.lead_time.dtypes

dtype('float64')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182877 entries, 0 to 182876
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  object 
 2   lead_time                       119490 non-null  float64
 3   arrival_date_year               64829 non-null   float64
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        101004 non-null  float64
 6   arrival_date_day_of_month       119271 non-null  float64
 7   stays_in_weekend_nights         119490 non-null  float64
 8   stays_in_week_nights            119490 non-null  float64
 9   adults                          119428 non-null  float64
 10  children                        69302 non-null   float64
 11  babies                          119431 non-null  float64
 12  meal                 